Notes:
* The data is for two years: 2023 and 2024
* The issuer_bank_name has only value of "My Favorite Bank". Possible to drop
* Null values seem to be reasonable.
    merchant_id = null when payment is automatic like salary  
    merchant_mcc = null whenever merchant_id = null  
    merchant_city = null Almost always when merchant_id = null  
    original amount = null if no conversion to other currency  
    wallet_type = null if regular card is used  
    pos_entry_mode = unclear what null signifies. When it is null the transaction is either  
    "P2P_OUT", "BILL_PAYMENT", "SALARY", "P2P_IN"  
* There is cases (~739) where we have merchant_city but no merchant_id

In [2]:
import polars as pl

data = pl.read_parquet("data/DECENTRATHON_3.0.parquet")

In [3]:
data.head()

transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
str,datetime[ns],i64,str,str,i64,i64,str,str,str,f64,f64,str,str,str,str,i64
"""ffb21bbd-fef1-4983-9774-3b01c8…",2023-08-12 23:14:19.043176,10000,"""09/26""","""My Favorite Bank""",50359,5499,"""Grocery & Food Markets""","""Shymkent""","""POS""",1237.66,null,"""KZT""","""KAZ""","""Contactless""","""Apple Pay""",0
"""acd754f9-bd4c-416d-babc-926050…",2023-08-12 17:07:08.641030,10000,"""09/26""","""My Favorite Bank""",50359,5499,"""Grocery & Food Markets""","""Shymkent""","""POS""",2328.03,null,"""KZT""","""KAZ""","""QR_Code""","""Bank's QR""",1
"""b023697b-eb35-40e6-af75-3586c6…",2023-08-12 08:45:32.946846,10000,"""09/26""","""My Favorite Bank""",50214,4814,"""Utilities & Bill Payments""","""Almaty""","""BILL_PAYMENT""",53871.78,null,"""KZT""","""KAZ""",null,null,2
"""d59d8537-1e48-41b5-91e1-dc7cf9…",2023-08-12 10:20:20.242510,10000,"""09/26""","""My Favorite Bank""",50279,8011,"""Pharmacies & Health""","""Shymkent""","""POS""",22501.56,null,"""KZT""","""KAZ""","""Contactless""","""Apple Pay""",3
"""85d2eef6-3b09-47a6-987f-f45ab4…",2023-08-12 00:42:00.556252,10000,"""09/26""","""My Favorite Bank""",50332,5499,"""Grocery & Food Markets""","""Almaty""","""POS""",1224.04,null,"""KZT""","""KAZ""","""QR_Code""","""Bank's QR""",4


In [4]:
data.describe()

statistic,transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
str,str,str,f64,str,str,f64,f64,str,str,str,f64,f64,str,str,str,str,f64
"""count""","""11536850""","""11536850""",1.153685e7,"""11536850""","""11536850""",9.066814e6,9.066814e6,"""11536850""","""9067553""","""11536850""",1.153685e7,5417.0,"""11536850""","""11536850""","""8883438""","""6863356""",1.153685e7
"""null_count""","""0""","""0""",0.0,"""0""","""0""",2.470036e6,2.470036e6,"""0""","""2469297""","""0""",0.0,1.1531433e7,"""0""","""0""","""2653412""","""4673494""",0.0
"""mean""",null,"""2024-04-23 10:50:42.587606""",11003.031321,null,null,50249.453106,5644.688338,null,null,null,20166.686882,74316.000809,null,null,null,null,5768424.5
"""std""",null,null,580.572998,null,null,143.985954,1030.229349,null,null,null,81305.720721,575082.106378,null,null,null,null,3.3304e6
"""min""","""0000013f-41fa-4e8e-b029-df002c…","""2023-01-01 00:30:12.551745""",10000.0,"""01/25""","""My Favorite Bank""",50000.0,3000.0,"""Clothing & Apparel""","""Aktobe""","""ATM_WITHDRAWAL""",100.0,0.21,"""AED""","""ARE""","""Chip""","""Apple Pay""",0.0
"""25%""",null,"""2023-12-21 14:19:07.908075""",10494.0,null,null,50126.0,5200.0,null,null,null,2140.45,28.8,null,null,null,null,2.884212e6
"""50%""",null,"""2024-05-19 05:29:53.788712""",10998.0,null,null,50248.0,5499.0,null,null,null,6070.66,164.78,null,null,null,null,5.768425e6
"""75%""",null,"""2024-09-17 17:55:15.468680""",11504.0,null,null,50375.0,5812.0,null,null,null,19259.91,2086.07,null,null,null,null,8.652637e6
"""max""","""fffffe8e-4a4f-4ba7-990c-19c995…","""2024-12-31 23:59:35.437993""",11999.0,"""12/29""","""My Favorite Bank""",50499.0,9311.0,"""Utilities & Bill Payments""","""Shymkent""","""SALARY""",3.4073e7,2.7190e7,"""UZS""","""UZB""","""Swipe""","""Samsung Pay""",1.1536849e7


In [5]:
data.null_count()

transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,2470036,2470036,0,2469297,0,0,11531433,0,0,2653412,4673494,0


In [7]:
total_transactions = data.get_column("transaction_amount_kzt").sum()

transactions_by_category = (data.group_by("mcc_category")
                                .agg(pl.col("transaction_amount_kzt").sum())
                                .with_columns(
                                    (pl.col("transaction_amount_kzt")/total_transactions).alias("share_in_ttl_trnsc")))


transactions_by_category.plot.bar(x="mcc_category", y = "share_in_ttl_trnsc")

alt.Chart(...)

In [8]:
transactions_by_type = (data.group_by("transaction_type")
                                .agg(pl.col("transaction_amount_kzt").sum())
                                .with_columns(
                                    (pl.col("transaction_amount_kzt")/total_transactions).alias("share_in_ttl_trnsc"))
                                .sort(by="share_in_ttl_trnsc", descending=True))

transactions_by_type.plot.bar(x="transaction_type", y = "share_in_ttl_trnsc")

alt.Chart(...)

In [ ]:
clustering_data = data.group_by("card_id").agg(
    (pl.col("transaction_timestamp").max()).alias("last_trans"),
    (pl.col("transaction_amount_kzt")).alias("total_trans"),
    (pl.col("transaction_timestamp").min().alias("firs_trans"))
)

In [ ]:
reference_date = pl.lit("2025-01-01").str.to_date()

clustering_data.with_columns(
   last_transac = (pl.col("transaction_timestamp") - reference_date).dt.total_minutes()
).describe()

statistic,card_id,transaction_timestamp,last_transac
str,f64,str,f64
"""count""",2000.0,"""2000""",2000.0
"""null_count""",0.0,"""0""",0.0
"""mean""",10999.5,"""2024-11-22 04:06:03.344196""",-955.534
"""std""",577.494589,null,2791.774849
"""min""",10000.0,"""2023-02-01 08:00:00""",-16792.0
"""25%""",10500.0,"""2024-12-31 19:30:41.010149""",-4.0
"""50%""",11000.0,"""2024-12-31 22:21:06.815404""",-1.0
"""75%""",11499.0,"""2024-12-31 23:21:09.397707""",0.0
"""max""",11999.0,"""2024-12-31 23:59:35.437993""",0.0


In [38]:
pos_transactions = data.pivot(
    values="transaction_amount_kzt",
    index="card_id",
    on="pos_entry_mode",
    aggregate_function="sum").rename(lambda c: f"pos_{c}" if c !="card_id" else c)

In [39]:
pos_transactions

card_id,pos_Contactless,pos_QR_Code,pos_null,pos_ECOM,pos_Swipe,pos_Chip
i64,f64,f64,f64,f64,f64,f64
10000,3.5213e7,3.2864e7,5.0410e7,2.5176e7,3.4890e6,2.3184e7
10001,5.0784e6,4.1942e6,3.9633e6,1.0824e7,51168.43,4.6520e6
10002,1.175e6,652721.12,3.7907e6,1.2838e7,null,1.181e6
10003,5.3083e6,3.3663e6,7.5315e6,1.4122e6,106201.04,3.7400e6
10004,1.8393e6,556085.74,5.2359e6,403911.39,13054.46,1.7404e6
…,…,…,…,…,…,…
11995,null,null,1.0253e7,null,null,null
11996,1.5201e8,1.5905e8,5.37422e6,4.5363e7,1.5435e7,7.5829e7
11997,4.8508e6,3.8601e6,2.7950e7,3.3509e7,199516.34,4861064.8
